# Wind Turbine EDA

In this notebook, we take data from USGS Wind Turbine database and do data preprocessing to get the turbines that are within the San Francisco Bay Area and Chicago area. We will also impute missing data that we need as necessary.

USGS Wind Turbine Database: https://eerscmap.usgs.gov/uswtdb/

The particular shape file was provided to us courtesy of Prof. Fernanda Lopez-Ornelas.

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gdf = gpd.read_file('/content/drive/MyDrive/CS 663 - Project/Datasets/uswtdbSHP/uswtdb_v6_1_20231128.shp')
gdf

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
0,3123985.0,40-098167,2021-WTW-7918-OE,-9999.0,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,180.1,0.0,-9999.0,3.0,3.0,2023-01-04,Digital Globe,-99.787033,36.501724,POINT (-99.78703 36.50172)
1,3123544.0,40-097504,2021-WTW-7909-OE,-9999.0,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,180.1,0.0,-9999.0,3.0,3.0,2023-01-04,Digital Globe,-99.725624,36.437126,POINT (-99.72562 36.43713)
2,3123887.0,40-097762,2021-WTW-7895-OE,-9999.0,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,180.1,0.0,-9999.0,3.0,3.0,2023-01-04,Digital Globe,-99.769722,36.444931,POINT (-99.76972 36.44493)
3,3123765.0,40-097546,2021-WTW-7863-OE,-9999.0,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,180.1,0.0,-9999.0,3.0,3.0,2023-01-04,Digital Globe,-99.807060,36.513935,POINT (-99.80706 36.51393)
4,3123814.0,40-097528,2021-WTW-7897-OE,-9999.0,65511.0,OK,Ellis County,40045,25 Mile Creek,2022.0,...,180.1,0.0,-9999.0,3.0,3.0,2023-01-04,Digital Globe,-99.758476,36.444984,POINT (-99.75848 36.44498)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73347,3124478.0,40-098732,2021-WTW-794-OE,-9999.0,-9999.0,OK,Woods County,40151,unknown Woods County,2021.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2022-10-17,Digital Globe,-98.633461,36.500687,POINT (-98.63346 36.50069)
73348,3053232.0,36-120351,2015-WTE-5788-OE,-9999.0,-9999.0,NY,Wyoming County,36121,unknown Wyoming County,2016.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2017-05-20,Digital Globe,-78.187935,42.740818,POINT (-78.18793 42.74082)
73349,3040944.0,48-024978,2011-WTW-352-OE,41364.0,-9999.0,TX,Young County,48503,unknown Young County 1,2011.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-07-16,Digital Globe,-98.551094,33.093292,POINT (-98.55109 33.09329)
73350,3055918.0,08-072237,2015-WTW-9995-OE,-9999.0,-9999.0,CO,Yuma County,08125,unknown Yuma County,2016.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2017-05-17,Digital Globe,-102.716949,40.037548,POINT (-102.71695 40.03755)


In [4]:
import ee
ee.Authenticate()

In [5]:
ee.Initialize(project='project1-1531690201568')

In [6]:
import geemap

In [7]:
feature_collection = ee.FeatureCollection('projects/project1-1531690201568/assets/uswtdbSHP')

The shape file was uploaded to Google Earth Engine. It was viewable using geemap in Colab with below code but not anymore after downloading this notebook. For reference, see slide 10 of Final Presentation Google Slide to see what the map would look like.

In [8]:
Map = geemap.Map()
Map.addLayer(feature_collection)
bbox = ee.Geometry.BBox(-88, 41, -87, 42)
bbox2 = ee.Geometry.BBox(-124, 36, -120, 40)
# Apply the bounds method to the BBox object.
bbox_bounds = bbox.bounds()
bbox2_bounds = bbox2.bounds()

# Print the result.
# display('bbox.bounds(...) =', bbox_bounds)

# Display relevant geometries on the map.
Map.add_layer(bbox, {'color': 'red', 'opacity': 0.2}, 'Geometry [red]: bbox')
Map.add_layer(bbox2, {'color': 'blue', 'opacity': 0.2}, 'Geometry [blue]: bbox2')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 73352 entries, 0 to 73351
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   case_id     73352 non-null  float64 
 1   faa_ors     73352 non-null  object  
 2   faa_asn     73352 non-null  object  
 3   usgs_pr_id  73352 non-null  float64 
 4   eia_id      73352 non-null  float64 
 5   t_state     73352 non-null  object  
 6   t_county    73352 non-null  object  
 7   t_fips      73352 non-null  object  
 8   p_name      73352 non-null  object  
 9   p_year      73352 non-null  float64 
 10  p_tnum      73352 non-null  float64 
 11  p_cap       73352 non-null  float64 
 12  t_manu      73352 non-null  object  
 13  t_model     73352 non-null  object  
 14  t_cap       73352 non-null  float64 
 15  t_hh        73352 non-null  float64 
 16  t_rd        73352 non-null  float64 
 17  t_rsa       73352 non-null  float64 
 18  t_ttlh      73352 non-null  float64 
 

In [10]:
gdf.describe()

,case_id,usgs_pr_id,eia_id,p_year,p_tnum,p_cap,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,xlong,ylat
count,7.335200e+04,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000,73352.000000
mean,3.064921e+06,9143.560721,54559.756776,1923.012897,102.600147,-167.545222,1494.215822,-513.716593,-447.817464,7200.593956,-467.850522,0.091463,-8899.807831,2.833297,2.963505,-99.893278,38.409815
std,3.580804e+04,21149.517129,16742.214820,1033.422009,91.072151,1867.864272,2696.779018,2378.125914,2286.048338,5386.726523,2391.452757,0.288268,3464.375530,0.498714,0.263510,10.985380,5.415445
min,3.000001e+06,-9999.000000,-9999.000000,-9999.000000,1.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.000000,-9999.000000,1.000000,1.000000,-171.713074,13.389381
25%,3.035070e+06,-9999.000000,56645.000000,2009.000000,56.000000,99.000000,1500.000000,80.000000,82.000000,5281.020000,119.500000,0.000000,-9999.000000,3.000000,3.000000,-102.811243,34.460477
50%,3.055180e+06,668.500000,57802.000000,2013.000000,84.000000,161.000000,2000.000000,80.000000,100.000000,7853.980000,130.100000,0.000000,-9999.000000,3.000000,3.000000,-99.121380,38.956955
75%,3.097572e+06,29148.250000,60689.000000,2019.000000,120.000000,223.500000,2400.000000,89.000000,116.000000,10568.320000,148.100000,0.000000,-9999.000000,3.000000,3.000000,-95.217344,42.732819
max,3.131501e+06,49135.000000,66321.000000,2023.000000,713.000000,1055.600000,6000.000000,137.000000,162.000000,20611.990000,205.400000,1.000000,2021.000000,3.000000,3.000000,144.722656,66.839905


In [11]:
## Chicago area

long = gdf[gdf.xlong <= -87]
long = long[long.xlong >= -88]
lat =  long[long.ylat <= 42]
chicago_area = lat[lat.ylat >= 41]
chicago_area

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
4340,3074349.0,missing,missing,48121.0,-9999.0,IL,Cook County,17031,Big Windy,2009.0,...,34.4,0.0,-9999.0,2.0,3.0,2017-08-30,Digital Globe,-87.865189,41.955463,POINT (-87.86519 41.95546)
37302,3038416.0,17-024050,2011-WTE-2999-OE,48128.0,-9999.0,IL,Kankakee County,17091,MUTI Wind Turbine,2010.0,...,-9999.0,0.0,-9999.0,2.0,3.0,2020-11-28,Digital Globe,-87.851395,41.095390,POINT (-87.85139 41.09539)
40193,3053439.0,17-029046,2015-WTE-741-OE,-9999.0,-9999.0,IL,Cook County,17031,Method Manufacturing Plant,2015.0,...,68.9,0.0,-9999.0,3.0,3.0,2017-06-05,Digital Globe,-87.601456,41.698063,POINT (-87.60146 41.69806)
44130,3074425.0,18-025755,2010-WTE-1327-OE,48137.0,-9999.0,IN,Newton County,18111,North Newton School Corporation,2012.0,...,98.0,0.0,-9999.0,3.0,3.0,2016-11-15,Digital Globe,-87.426270,41.032585,POINT (-87.42627 41.03258)
48661,3053255.0,17-028869,2015-WTE-6194-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.956772,41.032673,POINT (-87.95677 41.03267)
48666,3050056.0,17-029664,2014-WTE-7183-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.989967,41.010151,POINT (-87.98997 41.01015)
48679,3049908.0,17-029345,2014-WTE-6723-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.971146,41.025143,POINT (-87.97115 41.02514)
48699,3050054.0,17-029682,2014-WTE-7181-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.995171,41.017593,POINT (-87.99517 41.01759)
48702,3049858.0,17-029681,2014-WTE-6484-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.953682,41.027992,POINT (-87.95368 41.02799)
48704,3053256.0,17-028892,2015-WTE-6195-OE,-9999.0,58898.0,IL,Kankakee County,17091,Pilot Hill,2015.0,...,130.1,0.0,-9999.0,3.0,3.0,2017-04-13,Digital Globe,-87.957855,41.025230,POINT (-87.95785 41.02523)


In [12]:
chicago_area.t_model.value_counts()

t_model
GE1.7-100    17
NM19          1
E-3120        1
NM48          1
PW56-900      1
47-750        1
Name: count, dtype: int64

In [13]:
chicago_area.t_manu.value_counts()

t_manu
GE Wind        17
NEG Micon       2
Endurance       1
PowerWind       1
Aeronautica     1
Name: count, dtype: int64

In [14]:
## SF Bay Area

long = gdf[gdf.xlong <= -120]
long = long[long.xlong >= -124]
lat =  long[long.ylat <= 40]
SF_area = lat[lat.ylat >= 36]
SF_area

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
1402,3048446.0,06-037710,2013-WTW-7671-OE,-9999.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2014.0,...,121.3,0.0,-9999.0,3.0,3.0,2017-08-19,Digital Globe,-122.092079,38.233505,POINT (-122.09208 38.23351)
1403,3040085.0,06-022141,2011-WTW-11924-OE,13315.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2011.0,...,103.6,0.0,-9999.0,3.0,3.0,2020-08-27,Digital Globe,-122.089287,38.232296,POINT (-122.08929 38.23230)
7998,3036014.0,06-020764,2010-WTW-12669-OE,12004.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.644188,37.796196,POINT (-121.64419 37.79620)
7999,3020634.0,06-020723,2006-AWP-4981-OE,12027.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2020-03-03,Digital Globe,-121.666084,37.801395,POINT (-121.66608 37.80139)
8000,3020645.0,06-020776,2006-AWP-5001-OE,12032.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.653885,37.802994,POINT (-121.65389 37.80299)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,3127335.0,06-260894,2018-WTW-13406-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-06-14,Digital Globe,-121.808594,38.072895,POINT (-121.80859 38.07290)
72374,3127360.0,06-260889,2018-WTW-13400-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.825356,38.083965,POINT (-121.82536 38.08396)
72375,3127467.0,06-260901,2018-WTW-13399-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.829010,38.090183,POINT (-121.82901 38.09018)
72376,3127534.0,06-260898,2018-WTW-13396-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.750938,38.112133,POINT (-121.75094 38.11213)


In [15]:
SF_area.t_model.value_counts().sort_values()

t_model
29-STALL-225      1
108               1
GE 1.5-77         1
GE 2.7-116        2
GE2.82-127        2
GE1.85-82.5       4
GE2.3-116        19
GE2.7-116        21
V47-0.66         31
V90-1.8          31
SWT-2.3-101      34
MWT62/1.0        41
SWT-2.3-93       50
GE1.7-100        51
V90-3.0          53
NTK65            73
V80-1.8          90
GE1.5-77        108
MM92            175
missing         314
Name: count, dtype: int64

Here, we start to discover missing values and impute them.

In [16]:
SF_area[SF_area.t_model == 'missing'].t_manu.unique()

array(['missing', 'Bonus'], dtype=object)

In [17]:
SF_area[SF_area.t_manu == 'missing'].t_model.unique()

array(['missing'], dtype=object)

In [18]:
SF_area.t_manu.value_counts()

t_manu
GE Wind       208
Vestas        205
missing       182
REpower       175
Bonus         132
Siemens        84
Nordtank       73
Mitsubishi     41
Wincon          1
Norwin          1
Name: count, dtype: int64

In [19]:
SF_area[SF_area.t_model == 'missing'].p_name.unique()

array(['Dinosaur Point', 'Patterson Pass', 'unknown Alameda County',
       'unknown Solano County'], dtype=object)

In [20]:
SF_area.t_cap.value_counts()

t_cap
 65.0      205
 105.0     162
 1800.0    121
 1500.0    109
 2300.0    103
 2050.0    100
 2000.0     75
 3000.0     53
 1700.0     51
 1000.0     41
 660.0      31
 2700.0     23
-9999.0     21
 1850.0      4
 2820.0      2
 225.0       1
Name: count, dtype: int64

In [21]:
SF_area.t_conf_atr.value_counts()

t_conf_atr
3.0    844
1.0    182
2.0     76
Name: count, dtype: int64

In [22]:
chicago_area.describe()

,case_id,usgs_pr_id,eia_id,p_year,p_tnum,p_cap,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,xlong,ylat
count,2.200000e+01,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.0,22.0,22.000000,22.0,22.000000,22.000000
mean,3.051826e+06,-545.227273,43239.590909,2014.181818,79.818182,135.416364,1423.090909,-384.045455,-369.504545,5900.305455,-341.477273,0.0,-9999.0,2.909091,3.0,-87.911241,41.133015
std,8.375135e+03,21043.280810,29552.129880,1.842265,43.751067,74.895136,548.701229,2147.579490,2150.905322,4337.847749,2157.195401,0.0,0.0,0.294245,0.0,0.151995,0.283328
min,3.037690e+06,-9999.000000,-9999.000000,2009.000000,1.000000,0.100000,50.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.0,-9999.0,2.000000,3.0,-87.995827,41.005981
25%,3.049908e+06,-9999.000000,58898.000000,2015.000000,103.000000,175.100000,1700.000000,80.000000,100.000000,7853.980000,130.100000,0.0,-9999.0,3.000000,3.0,-87.990179,41.017451
50%,3.050056e+06,-9999.000000,58898.000000,2015.000000,103.000000,175.100000,1700.000000,80.000000,100.000000,7853.980000,130.100000,0.0,-9999.0,3.000000,3.0,-87.974419,41.023823
75%,3.053224e+06,-9999.000000,58898.000000,2015.000000,103.000000,175.100000,1700.000000,80.000000,100.000000,7853.980000,130.100000,0.0,-9999.0,3.000000,3.0,-87.954454,41.031437
max,3.074425e+06,48137.000000,58898.000000,2015.000000,103.000000,175.100000,1700.000000,80.000000,100.000000,7853.980000,130.100000,0.0,-9999.0,3.000000,3.0,-87.426270,41.955463


In [23]:
SF_area[SF_area.t_model != 'missing'].p_name.unique()

array(['Anheuser-Busch Fairfield', 'Buena Vista', 'Cemex Madison',
       'City of Soledad Water Reclamation Facility',
       'Correctional Training Facility', 'Diablo Winds', 'Dinosaur Point',
       'Dole Fresh Vegetables', 'Golden Hills', 'Golden Hills North',
       'High Winds', 'Mann Packing', 'McEvoy Ranch', 'Montezuma',
       'Montezuma Winds II', 'Patterson Wind', 'Safeway Tracy',
       'Salinas Valley State Prison', 'Scheid Vineyards', 'Shiloh II',
       'Shiloh III', 'Shiloh IV', 'Shiloh Wind Project', 'Solano Phase 3',
       'Solano Phase IIA', 'Solano Wind Project',
       'Summit Wind Repowering', 'Superior Farms', 'Taylor Farms',
       'Teichert Aggregates', 'Vasco (repower)'], dtype=object)

In [24]:
SF_area.describe()

,case_id,usgs_pr_id,eia_id,p_year,p_tnum,p_cap,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,xlong,ylat
count,1.102000e+03,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.0,1102.0,1102.000000,1102.000000,1102.000000,1102.000000
mean,3.030284e+06,9878.862976,35288.662432,1805.529946,93.401996,-117.963516,1067.224138,-1607.930399,-1601.227768,2323.768521,-1578.316243,0.0,-9999.0,2.600726,2.967332,-121.650851,37.815948
std,2.703627e+04,7033.291216,29146.335976,1521.877445,60.386320,1379.023647,1814.691580,3746.180420,3749.232948,6140.122599,3759.506698,0.0,0.0,0.755434,0.253628,0.218109,0.395585
min,3.000025e+06,-9999.000000,-9999.000000,-9999.000000,1.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.0,-9999.0,1.000000,1.000000,-122.679672,36.274509
25%,3.008400e+06,11155.250000,10005.000000,1988.000000,48.000000,17.030000,105.000000,24.600000,16.000000,201.060000,32.600000,0.0,-9999.0,3.000000,3.000000,-121.805158,37.697230
50%,3.020648e+06,12270.500000,56362.000000,2006.000000,90.000000,63.000000,1500.000000,62.500000,77.000000,4656.630000,101.800000,0.0,-9999.0,3.000000,3.000000,-121.704685,37.801013
75%,3.041045e+06,13020.750000,57618.000000,2011.000000,166.000000,127.800000,2050.000000,80.000000,92.500000,6720.060000,125.000000,0.0,-9999.0,3.000000,3.000000,-121.612450,38.139743
max,3.129272e+06,13315.000000,65525.000000,2022.000000,201.000000,162.000000,3000.000000,90.000000,127.000000,12667.690000,152.400000,0.0,-9999.0,3.000000,3.000000,-121.163284,38.687595


In [25]:
## Need to map out where the missing turbines are
missing_t_manu = SF_area[SF_area.t_manu == 'missing']
missing_t_models = SF_area[SF_area.t_model == 'missing']

missing_t_models

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
16871,3004395.0,missing,missing,12251.0,10005.0,CA,Merced County,06047,Dinosaur Point,1988.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-09-19,Digital Globe,-121.197510,37.036377,POINT (-121.19751 37.03638)
16872,3001809.0,missing,missing,12181.0,10005.0,CA,Merced County,06047,Dinosaur Point,1988.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-09-19,Digital Globe,-121.168755,37.039700,POINT (-121.16876 37.03970)
16873,3003395.0,missing,missing,12256.0,10005.0,CA,Merced County,06047,Dinosaur Point,1988.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-09-19,Digital Globe,-121.197876,37.038147,POINT (-121.19788 37.03815)
16874,3007841.0,missing,missing,12279.0,10005.0,CA,Merced County,06047,Dinosaur Point,1988.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-09-19,Digital Globe,-121.173187,37.044495,POINT (-121.17319 37.04449)
16875,3002781.0,missing,missing,12300.0,10005.0,CA,Merced County,06047,Dinosaur Point,1988.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2018-09-19,Digital Globe,-121.180084,37.051395,POINT (-121.18008 37.05139)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,3127335.0,06-260894,2018-WTW-13406-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-06-14,Digital Globe,-121.808594,38.072895,POINT (-121.80859 38.07290)
72374,3127360.0,06-260889,2018-WTW-13400-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.825356,38.083965,POINT (-121.82536 38.08396)
72375,3127467.0,06-260901,2018-WTW-13399-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.829010,38.090183,POINT (-121.82901 38.09018)
72376,3127534.0,06-260898,2018-WTW-13396-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.750938,38.112133,POINT (-121.75094 38.11213)


In [26]:
condensed_missing_t_models = missing_t_models.loc[:,['p_name','t_manu','t_model','t_cap','t_hh']]
condensed_missing_t_models.p_name.value_counts()

p_name
Dinosaur Point            161
Patterson Pass            132
unknown Solano County      20
unknown Alameda County      1
Name: count, dtype: int64

In [27]:
SF_area.p_name.value_counts()

p_name
Dinosaur Point                                162
Patterson Pass                                132
Shiloh Wind Project                           106
High Winds                                     90
Shiloh II                                      75
Patterson Wind                                 73
Solano Phase 3                                 55
Shiloh IV                                      50
Shiloh III                                     50
Golden Hills                                   48
Buena Vista                                    38
Montezuma Winds II                             34
Vasco (repower)                                34
Diablo Winds                                   31
Summit Wind Repowering                         23
Solano Wind Project                            21
unknown Solano County                          20
Golden Hills North                             19
Montezuma                                      16
Solano Phase IIA                           

In [28]:
Dinosaur_Point_Project = SF_area[SF_area['p_name'] == 'Dinosaur Point']
Dinosaur_Point_Project.loc[:,['p_name','t_manu','t_model','t_cap','t_hh']]

,p_name,t_manu,t_model,t_cap,t_hh
16871,Dinosaur Point,missing,missing,105.0,-9999.0
16872,Dinosaur Point,missing,missing,105.0,-9999.0
16873,Dinosaur Point,missing,missing,105.0,-9999.0
16874,Dinosaur Point,missing,missing,105.0,-9999.0
16875,Dinosaur Point,missing,missing,105.0,-9999.0
...,...,...,...,...,...
17028,Dinosaur Point,missing,missing,105.0,-9999.0
17029,Dinosaur Point,missing,missing,105.0,-9999.0
17030,Dinosaur Point,missing,missing,105.0,-9999.0
17031,Dinosaur Point,missing,missing,105.0,-9999.0


In [29]:
print("Dinosaur_Point_Project.t_manu.value_counts(): \n", Dinosaur_Point_Project.t_manu.value_counts())
print("Dinosaur_Point_Project.t_model.value_counts(): \n", Dinosaur_Point_Project.t_model.value_counts())
print("Dinosaur_Point_Project.t_cap.value_counts(): \n", Dinosaur_Point_Project.t_cap.value_counts())
print("Dinosaur_Point_Project.t_hh.value_counts(): \n", Dinosaur_Point_Project.t_hh.value_counts())
print("Dinosaur_Point_Project.t_conf_atr.value_counts(): \n", Dinosaur_Point_Project.t_conf_atr.value_counts())

Dinosaur_Point_Project.t_manu.value_counts(): 
 t_manu
missing    161
Wincon       1
Name: count, dtype: int64
Dinosaur_Point_Project.t_model.value_counts(): 
 t_model
missing    161
108          1
Name: count, dtype: int64
Dinosaur_Point_Project.t_cap.value_counts(): 
 t_cap
105.0    162
Name: count, dtype: int64
Dinosaur_Point_Project.t_hh.value_counts(): 
 t_hh
-9999.0    162
Name: count, dtype: int64
Dinosaur_Point_Project.t_conf_atr.value_counts(): 
 t_conf_atr
1.0    161
2.0      1
Name: count, dtype: int64


In [30]:
Patterson_Pass_Project = SF_area[SF_area['p_name'] == 'Patterson Pass']
Patterson_Pass_Project.p_year.value_counts()

p_year
1985.0    132
Name: count, dtype: int64

In [31]:
# Patterson Pass Wind farm decommissioned?
# https://www.gem.wiki/Patterson_Pass_wind_farm

# remove from SF_area

SF_area_active = SF_area[SF_area['p_name'] != 'Patterson Pass']
SF_area_active

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
1402,3048446.0,06-037710,2013-WTW-7671-OE,-9999.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2014.0,...,121.3,0.0,-9999.0,3.0,3.0,2017-08-19,Digital Globe,-122.092079,38.233505,POINT (-122.09208 38.23351)
1403,3040085.0,06-022141,2011-WTW-11924-OE,13315.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2011.0,...,103.6,0.0,-9999.0,3.0,3.0,2020-08-27,Digital Globe,-122.089287,38.232296,POINT (-122.08929 38.23230)
7998,3036014.0,06-020764,2010-WTW-12669-OE,12004.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.644188,37.796196,POINT (-121.64419 37.79620)
7999,3020634.0,06-020723,2006-AWP-4981-OE,12027.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2020-03-03,Digital Globe,-121.666084,37.801395,POINT (-121.66608 37.80139)
8000,3020645.0,06-020776,2006-AWP-5001-OE,12032.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.653885,37.802994,POINT (-121.65389 37.80299)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,3127335.0,06-260894,2018-WTW-13406-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-06-14,Digital Globe,-121.808594,38.072895,POINT (-121.80859 38.07290)
72374,3127360.0,06-260889,2018-WTW-13400-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.825356,38.083965,POINT (-121.82536 38.08396)
72375,3127467.0,06-260901,2018-WTW-13399-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.829010,38.090183,POINT (-121.82901 38.09018)
72376,3127534.0,06-260898,2018-WTW-13396-OE,-9999.0,-9999.0,CA,Solano County,06095,unknown Solano County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,1.0,2023-05-25,Digital Globe,-121.750938,38.112133,POINT (-121.75094 38.11213)


In [32]:
SF_area_active[SF_area_active['p_name'] == 'Dinosaur Point'].index

Index([16871, 16872, 16873, 16874, 16875, 16876, 16877, 16878, 16879, 16880,
       ...
       17023, 17024, 17025, 17026, 17027, 17028, 17029, 17030, 17031, 17032],
      dtype='int64', length=162)

In [33]:
for row in SF_area_active[SF_area_active['p_name'] == 'Dinosaur Point'].index:
  SF_area_active.loc[row,'t_manu'] = 'Wincon'
  SF_area_active.loc[row,'t_model'] = 108

In [34]:
SF_area_active.t_model.value_counts()

t_model
MM92            175
108             162
GE1.5-77        108
V80-1.8          90
NTK65            73
V90-3.0          53
GE1.7-100        51
SWT-2.3-93       50
MWT62/1.0        41
SWT-2.3-101      34
V90-1.8          31
V47-0.66         31
GE2.7-116        21
missing          21
GE2.3-116        19
GE1.85-82.5       4
GE 2.7-116        2
GE2.82-127        2
29-STALL-225      1
GE 1.5-77         1
Name: count, dtype: int64

In [35]:
SF_area_active.t_manu.value_counts()

t_manu
GE Wind       208
Vestas        205
REpower       175
Wincon        162
Siemens        84
Nordtank       73
Mitsubishi     41
missing        21
Norwin          1
Name: count, dtype: int64

In [36]:
remaining_missing = SF_area_active[SF_area_active['t_manu'] == 'missing'].loc[:,['t_manu','t_model','p_name', 'p_cap', 'p_tnum', 't_cap', 't_hh', 't_rd', 't_conf_atr']]
remaining_missing

,t_manu,t_model,p_name,p_cap,p_tnum,t_cap,t_hh,t_rd,t_conf_atr
70808,missing,missing,unknown Alameda County,-9999.0,1.0,-9999.0,-9999.0,-9999.0,1.0
72358,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72359,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72360,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72361,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72362,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72363,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72364,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72365,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0
72366,missing,missing,unknown Solano County,-9999.0,20.0,-9999.0,-9999.0,-9999.0,1.0


In [37]:
SF_area_active = SF_area_active[SF_area_active['t_manu'] != 'missing']
SF_area_active

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
1402,3048446.0,06-037710,2013-WTW-7671-OE,-9999.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2014.0,...,121.3,0.0,-9999.0,3.0,3.0,2017-08-19,Digital Globe,-122.092079,38.233505,POINT (-122.09208 38.23351)
1403,3040085.0,06-022141,2011-WTW-11924-OE,13315.0,59331.0,CA,Solano County,06095,Anheuser-Busch Fairfield,2011.0,...,103.6,0.0,-9999.0,3.0,3.0,2020-08-27,Digital Globe,-122.089287,38.232296,POINT (-122.08929 38.23230)
7998,3036014.0,06-020764,2010-WTW-12669-OE,12004.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.644188,37.796196,POINT (-121.64419 37.79620)
7999,3020634.0,06-020723,2006-AWP-4981-OE,12027.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2020-03-03,Digital Globe,-121.666084,37.801395,POINT (-121.66608 37.80139)
8000,3020645.0,06-020776,2006-AWP-5001-OE,12032.0,56446.0,CA,Contra Costa County,06013,Buena Vista,2006.0,...,85.6,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.653885,37.802994,POINT (-121.65389 37.80299)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66184,3039830.0,06-022144,2011-WTW-10310-OE,11999.0,57700.0,CA,Contra Costa County,06013,Vasco (repower),2011.0,...,130.5,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.709885,37.794193,POINT (-121.70988 37.79419)
66185,3039895.0,06-022125,2011-WTW-10978-OE,11992.0,57700.0,CA,Contra Costa County,06013,Vasco (repower),2011.0,...,130.5,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.701584,37.789993,POINT (-121.70158 37.78999)
66186,3039831.0,06-022145,2011-WTW-10311-OE,11991.0,57700.0,CA,Contra Costa County,06013,Vasco (repower),2011.0,...,130.5,0.0,-9999.0,3.0,3.0,2020-03-03,Digital Globe,-121.710487,37.788895,POINT (-121.71049 37.78889)
66187,3039880.0,06-022148,2011-WTW-10641-OE,12039.0,57700.0,CA,Contra Costa County,06013,Vasco (repower),2011.0,...,130.5,0.0,-9999.0,3.0,3.0,2012-01-01,NAIP,-121.726784,37.804794,POINT (-121.72678 37.80479)


In [38]:
SF_area_active.t_manu.value_counts()

t_manu
GE Wind       208
Vestas        205
REpower       175
Wincon        162
Siemens        84
Nordtank       73
Mitsubishi     41
Norwin          1
Name: count, dtype: int64

In [39]:
SF_area_active.t_model.value_counts()

t_model
MM92            175
108             162
GE1.5-77        108
V80-1.8          90
NTK65            73
V90-3.0          53
GE1.7-100        51
SWT-2.3-93       50
MWT62/1.0        41
SWT-2.3-101      34
V47-0.66         31
V90-1.8          31
GE2.7-116        21
GE2.3-116        19
GE1.85-82.5       4
GE2.82-127        2
GE 2.7-116        2
GE 1.5-77         1
29-STALL-225      1
Name: count, dtype: int64

In [40]:
SF_area_active[SF_area_active.t_model == 'GE 2.7-116']

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
61854,3111632.0,06-230600,2018-WTW-12885-OE,-9999.0,64326.0,CA,Alameda County,06001,Summit Wind Repowering,2021.0,...,148.0,0.0,-9999.0,3.0,3.0,2021-05-26,Digital Globe,-121.685707,37.736950,POINT (-121.68571 37.73695)
61859,3111191.0,06-230599,2018-WTW-12883-OE,-9999.0,64326.0,CA,Alameda County,06001,Summit Wind Repowering,2021.0,...,148.0,0.0,-9999.0,3.0,3.0,2021-04-11,Digital Globe,-121.692932,37.740356,POINT (-121.69293 37.74036)


In [41]:
SF_area_active.loc[61854, 't_model'] = 'GE2.7-116'
SF_area_active.loc[61859, 't_model'] = 'GE2.7-116'

In [42]:
SF_area_active[SF_area_active.t_model == 'GE 1.5-77']

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
63264,3061589.0,06-020792,2009-WTW-9229-OE,11946.0,57594.0,CA,San Joaquin County,06077,Teichert Aggregates,2010.0,...,118.5,0.0,-9999.0,3.0,3.0,2017-08-12,Digital Globe,-121.35656,37.616653,POINT (-121.35656 37.61665)


In [43]:
SF_area_active.loc[63264, 't_model'] = 'GE1.5-77'

In [44]:
SF_area_active.t_model.value_counts()

t_model
MM92            175
108             162
GE1.5-77        109
V80-1.8          90
NTK65            73
V90-3.0          53
GE1.7-100        51
SWT-2.3-93       50
MWT62/1.0        41
SWT-2.3-101      34
V90-1.8          31
V47-0.66         31
GE2.7-116        23
GE2.3-116        19
GE1.85-82.5       4
GE2.82-127        2
29-STALL-225      1
Name: count, dtype: int64

In [45]:
SF_area_active.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 949 entries, 1402 to 66188
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   case_id     949 non-null    float64 
 1   faa_ors     949 non-null    object  
 2   faa_asn     949 non-null    object  
 3   usgs_pr_id  949 non-null    float64 
 4   eia_id      949 non-null    float64 
 5   t_state     949 non-null    object  
 6   t_county    949 non-null    object  
 7   t_fips      949 non-null    object  
 8   p_name      949 non-null    object  
 9   p_year      949 non-null    float64 
 10  p_tnum      949 non-null    float64 
 11  p_cap       949 non-null    float64 
 12  t_manu      949 non-null    object  
 13  t_model     949 non-null    object  
 14  t_cap       949 non-null    float64 
 15  t_hh        949 non-null    float64 
 16  t_rd        949 non-null    float64 
 17  t_rsa       949 non-null    float64 
 18  t_ttlh      949 non-null    float64 
 19  

In [49]:
SF_area_active[SF_area_active['t_model'] == 108].loc[:,['t_manu', 't_model', 't_cap', 't_rd', 't_rsa', 't_ttlh', 't_hh']]

,t_manu,t_model,t_cap,t_rd,t_rsa,t_ttlh,t_hh
16871,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
16872,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
16873,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
16874,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
16875,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...
17028,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
17029,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
17030,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0
17031,Wincon,108,105.0,-9999.0,-9999.0,-9999.0,-9999.0


In [57]:
SF_area_active[SF_area_active['t_model'] == 108].loc[:,['t_manu', 't_model', 't_cap', 't_rd', 't_rsa', 't_ttlh', 't_hh']].t_cap.value_counts()

t_cap
105.0    162
Name: count, dtype: int64

In [51]:
chicago_area[chicago_area['t_model'] == 'NM19'].loc[:,['t_manu', 't_model', 't_cap', 't_rd', 't_rsa', 't_ttlh', 't_hh', 't_conf_atr']]

,t_manu,t_model,t_cap,t_rd,t_rsa,t_ttlh,t_hh,t_conf_atr
4340,NEG Micon,NM19,108.0,18.9,280.55,34.4,25.0,2.0


In [52]:
SF_area_active[SF_area_active['t_model'] == 'GE2.7-116'].loc[:,['t_manu', 't_model', 't_cap', 't_rd', 't_rsa', 't_ttlh', 't_hh', 't_conf_atr']]

,t_manu,t_model,t_cap,t_rd,t_rsa,t_ttlh,t_hh,t_conf_atr
61852,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61853,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61854,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.0,90.0,3.0
61855,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61856,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61857,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61858,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61859,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.0,90.0,3.0
61860,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0
61861,GE Wind,GE2.7-116,2700.0,116.0,10568.32,148.1,90.0,3.0


In [53]:
SF_area_active[SF_area_active['t_manu'] == 'Bonus'].loc[:,['t_manu', 't_model', 't_cap', 't_rd', 't_rsa', 't_ttlh', 't_hh', 't_conf_atr']]

,t_manu,t_model,t_cap,t_rd,t_rsa,t_ttlh,t_hh,t_conf_atr


In [73]:
SF_area_active.describe()

,case_id,usgs_pr_id,eia_id,p_year,p_tnum,p_cap,t_cap,t_hh,t_rd,t_rsa,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,xlong,ylat
count,9.490000e+02,949.000000,949.000000,949.000000,949.000000,949.000000,949.000000,949.000000,949.000000,949.000000,949.000000,949.0,949.0,949.000000,949.0,949.000000,949.000000
mean,3.031594e+06,10152.268704,42590.045311,2003.794521,80.080084,82.463609,1451.506849,-1649.322972,-1640.343519,2891.709157,-1616.047102,0.0,-9999.0,2.580611,3.0,-121.652916,37.825777
std,2.389420e+04,6958.401886,24540.484833,10.366233,48.193964,56.789134,907.825708,3790.290012,3794.401950,6282.650600,3805.454889,0.0,0.0,0.763820,0.0,0.233647,0.420910
min,3.000025e+06,-9999.000000,-9999.000000,1985.000000,1.000000,0.225000,65.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.0,-9999.0,1.000000,3.0,-122.679672,36.274509
25%,3.011352e+06,12005.000000,10005.000000,2003.000000,38.000000,17.030000,660.000000,55.000000,47.000000,1734.940000,83.500000,0.0,-9999.0,3.000000,3.0,-121.814590,37.704433
50%,3.029496e+06,12438.000000,56362.000000,2006.000000,75.000000,78.200000,1800.000000,78.500000,80.000000,5026.550000,118.600000,0.0,-9999.0,3.000000,3.0,-121.752884,38.089195
75%,3.041500e+06,13078.000000,57701.000000,2012.000000,100.000000,150.000000,2050.000000,80.000000,92.500000,6720.060000,125.000000,0.0,-9999.0,3.000000,3.0,-121.615852,38.144096
max,3.123818e+06,13315.000000,65525.000000,2022.000000,166.000000,162.000000,3000.000000,90.000000,127.000000,12667.690000,152.400000,0.0,-9999.0,3.000000,3.0,-121.163284,38.687595


After data is cleaned, we export to csv for future reference.

In [83]:
SF_area_active.to_csv('/content/drive/My Drive/CS 663 - Project/SF_Area_Turbines.csv')

In [84]:
chicago_area.to_csv('/content/drive/My Drive/CS 663 - Project/Chicago_Turbines.csv')

Below is just draft code.

In [ ]:
long = gdf[gdf.xlong <= -116]
long = long[long.xlong >= -120]
lat =  long[long.ylat <= 36]
lat = lat[lat.ylat >= 32]
lat

,case_id,faa_ors,faa_asn,usgs_pr_id,eia_id,t_state,t_county,t_fips,p_name,p_year,...,t_ttlh,retrofit,retrofit_y,t_conf_atr,t_conf_loc,t_img_date,t_img_srce,xlong,ylat,geometry
60,3072704.0,missing,missing,5146.0,52161.0,CA,Kern County,06029,251 Wind,1987.0,...,-9999.0,0.0,-9999.0,2.0,3.0,2018-05-08,Digital Globe,-118.364197,35.077644,POINT (-118.36420 35.07764)
61,3072661.0,missing,missing,5149.0,52161.0,CA,Kern County,06029,251 Wind,1987.0,...,-9999.0,0.0,-9999.0,2.0,3.0,2018-05-08,Digital Globe,-118.363762,35.077908,POINT (-118.36376 35.07791)
62,3072695.0,missing,missing,5143.0,52161.0,CA,Kern County,06029,251 Wind,1987.0,...,-9999.0,0.0,-9999.0,2.0,3.0,2018-05-08,Digital Globe,-118.364410,35.077435,POINT (-118.36441 35.07744)
304,3025453.0,06-020005,2007-AWP-2477-OE,2453.0,-9999.0,CA,Kern County,06029,Alite Wind Farm,2008.0,...,125.0,0.0,-9999.0,3.0,3.0,2021-02-20,Digital Globe,-118.343689,35.031494,POINT (-118.34369 35.03149)
305,3025446.0,06-003251,2007-AWP-2423-OE,2486.0,-9999.0,CA,Kern County,06029,Alite Wind Farm,2008.0,...,125.0,0.0,-9999.0,3.0,3.0,2020-01-16,Digital Globe,-118.345291,35.035194,POINT (-118.34529 35.03519)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,3126093.0,06-243014,2021-WTW-9163-OE,-9999.0,-9999.0,CA,Ventura County,06111,unknown Ventura County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2022-08-13,Digital Globe,-119.468391,33.230350,POINT (-119.46839 33.23035)
73177,3125951.0,06-243018,2021-WTW-9157-OE,-9999.0,-9999.0,CA,Ventura County,06111,unknown Ventura County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2022-08-13,Digital Globe,-119.476494,33.230885,POINT (-119.47649 33.23089)
73178,3126024.0,06-243015,2021-WTW-9159-OE,-9999.0,-9999.0,CA,Ventura County,06111,unknown Ventura County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2022-08-13,Digital Globe,-119.474289,33.232086,POINT (-119.47429 33.23209)
73179,3126021.0,06-243017,2021-WTW-9162-OE,-9999.0,-9999.0,CA,Ventura County,06111,unknown Ventura County,-9999.0,...,-9999.0,0.0,-9999.0,1.0,3.0,2022-08-13,Digital Globe,-119.469971,33.231026,POINT (-119.46997 33.23103)


In [ ]:
lat.t_model.value_counts()

missing        1746
V90-3.0         436
GE1.5-77        352
180             176
NTK65/13        134
               ... 
NM54              1
EW 1.5S           1
GE 1.5-70.5       1
V-112             1
108               1
Name: t_model, Length: 62, dtype: int64